In [1]:
import astropy.units as u
import astropy.constants as const
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

In [2]:
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator

color_map = {
    "green1": "#66c2a5",
    "orange": "#fc8d62",
    "blue": "#8da0cb",
    "pink": "#e78ac3",
    "green2": "#a6d854",
    "yellow": "#ffd92f",
    "brown": "#e5c494",
    "grey": "#b3b3b3",
}

In [3]:
import glob
import os
import sys
sys.path.append('../tools/')

from dust_extinction import calALambda
from spec_to_flux import spec_to_flux, spec_to_mag
from data_binning import data_binning
from visual import rcparams_format

rcparams_format(mpl.rcParams)

## Distance modulus

In [4]:
# SN metadata

z = 0.02736
z_unc = 0.00007
# t_max_mjd = 59723.65 # t_0 in salt3
t_max_mjd = 59722.75 # UVOT
ebv = 0.032
rv = 3.1

# tfl = 58972.46296299994


# Light curves

In [5]:
import pandas as pd
from astropy.table import Table

## ZTF

In [33]:
joj_ZTF = pd.read_csv('./ZTF22aajijjf_forced_fnu.csv')
# joj_ZTF = joj_ZTF[joj_ZTF['programid'] <= 3]
joj_ZTF['MJD'] = joj_ZTF['jd'] - 2400000.5
joj_ZTF['phase'] = (joj_ZTF['MJD'] - t_max_mjd) / (1 + z)
joj_ZTF['fnu_microJy'] = joj_ZTF['fnu_microJy'] * (1 + z)
joj_ZTF['fnu_microJy_unc'] = joj_ZTF['fnu_microJy_unc'] * (1 + z)
arg = (joj_ZTF['phase'] <= 150) & (joj_ZTF['phase'] >= -25)
joj_ZTF = joj_ZTF[arg]
joj_ZTF['mag'] = -2.5 * np.log10(joj_ZTF['fnu_microJy'] * 1e-6/3631)
joj_ZTF['magerr'] = 2.5/np.log(10) * joj_ZTF['fnu_microJy_unc']/joj_ZTF['fnu_microJy']
joj_ZTF['mag_err_u'] = -2.5 * np.log10((joj_ZTF['fnu_microJy'] - joj_ZTF['fnu_microJy_unc']) * 1e-6/3631) - joj_ZTF['mag']
joj_ZTF['mag_err_l'] = 2.5 * np.log10((joj_ZTF['fnu_microJy'] + joj_ZTF['fnu_microJy_unc']) * 1e-6/3631) + joj_ZTF['mag']
joj_ZTF_g = joj_ZTF[joj_ZTF['passband'] == 'ZTF_g'].assign(passband='ztfg')
joj_ZTF_r = joj_ZTF[joj_ZTF['passband'] == 'ZTF_r'].assign(passband='ztfr')
joj_ZTF_i = joj_ZTF[joj_ZTF['passband'] == 'ZTF_i'].assign(passband='ztfi')
joj_ZTF = pd.concat([joj_ZTF_g, joj_ZTF_r, joj_ZTF_i])

joj_ZTF_tex = {}
joj_ZTF_tex["time"] = joj_ZTF["MJD"]
joj_ZTF_tex["flux"] = joj_ZTF["fnu_microJy"]
joj_ZTF_tex["fluxerr"] = joj_ZTF["fnu_microJy_unc"]
joj_ZTF_tex["mag"] = joj_ZTF["mag"]
joj_ZTF_tex["magerr"] = joj_ZTF["magerr"]

joj_ZTF_tex = pd.DataFrame(joj_ZTF_tex)


filt = np.empty(len(joj_ZTF_tex), dtype=object)
for k in range(len(joj_ZTF)):
    filt[k] = "ZTF " + rf"${joj_ZTF['passband'].values[k].split('ztf')[-1]}$"
joj_ZTF_tex["filter"] = filt

joj_ZTF_tex["telescope"] = "P48/ZTF"
joj_ZTF_tex["system"] = "AB"

joj_ZTF_tex

/Users/chang/miniconda3/envs/astron/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/chang/miniconda3/envs/astron/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/chang/miniconda3/envs/astron/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,time,flux,fluxerr,mag,magerr,filter,telescope,system
986,59697.285602,4.239953,2.362115,22.331663,0.604873,ZTF $g$,P48/ZTF,AB
991,59699.441944,1.980350,3.302852,23.158211,1.810805,ZTF $g$,P48/ZTF,AB
992,59699.442882,-0.511017,3.243941,NaN,-6.892265,ZTF $g$,P48/ZTF,AB
994,59700.338357,-1.250067,3.232476,NaN,-2.807543,ZTF $g$,P48/ZTF,AB
995,59701.315486,2.467039,3.157259,22.919626,1.389500,ZTF $g$,P48/ZTF,AB
...,...,...,...,...,...,...,...,...
1085,59784.191366,94.182563,6.092168,18.965139,0.070230,ZTF $i$,P48/ZTF,AB
1090,59787.182245,78.916923,5.724211,19.157140,0.078753,ZTF $i$,P48/ZTF,AB
1091,59790.193032,69.371461,6.701406,19.297114,0.104884,ZTF $i$,P48/ZTF,AB
1094,59794.191389,64.409949,6.536666,19.377683,0.110186,ZTF $i$,P48/ZTF,AB


In [9]:
# joj_ZTF = joj_ZTF[~np.isnan(joj_ZTF['mag'])]
joj_ZTF_g = joj_ZTF[joj_ZTF['passband'] == 'ztfg']
joj_ZTF_r = joj_ZTF[joj_ZTF['passband'] == 'ztfr']
joj_ZTF_i = joj_ZTF[joj_ZTF['passband'] == 'ztfi']

## UVOT

In [42]:
joj_UVOT = pd.read_csv(
    "./UVOT_light_curve.dat",
    delimiter=" ",
    names=["time", "flux", "fluxerr", "mag", "magerr", "mag_unc_u", "mag_unc_l", "lim", "band"],
)
joj_UVOT["magerr"] = 2.5 / np.log(10) * (joj_UVOT["fluxerr"] / joj_UVOT["flux"])
joj_UVOT["telescope"] = "Swift/UVOT"
joj_UVOT["system"] = "AB"

joj_UVOT_u = joj_UVOT[joj_UVOT["band"] == "U"]
joj_UVOT_b = joj_UVOT[joj_UVOT["band"] == "B"]
joj_UVOT_v = joj_UVOT[joj_UVOT["band"] == "V"]
joj_UVOT_uvw1 = joj_UVOT[joj_UVOT["band"] == "UW1"]
joj_UVOT_uvw2 = joj_UVOT[joj_UVOT["band"] == "UW2"]
joj_UVOT_uvm2 = joj_UVOT[joj_UVOT["band"] == "UM2"]
joj_UVOT_u = joj_UVOT[joj_UVOT["band"] == "U"].assign(band="uvot::u")
joj_UVOT_b = joj_UVOT[joj_UVOT["band"] == "B"].assign(band="uvot::b")
joj_UVOT_v = joj_UVOT[joj_UVOT["band"] == "V"].assign(band="uvot::v")
joj_UVOT_uvw1 = joj_UVOT[joj_UVOT["band"] == "UW1"].assign(band="uvot::uvw1")
joj_UVOT_uvw2 = joj_UVOT[joj_UVOT["band"] == "UW2"].assign(band="uvot::uvw2")
joj_UVOT_uvm2 = joj_UVOT[joj_UVOT["band"] == "UM2"].assign(band="uvot::uvm2")

joj_UVOT_u["filter"] = "$u$"
joj_UVOT_b["filter"] = "$b$"
joj_UVOT_v["filter"] = "$v$"
joj_UVOT_uvw1["filter"] = "$uvw1$"
joj_UVOT_uvw2["filter"] = "$uvw2$"
joj_UVOT_uvm2["filter"] = "$uvm2$"

joj_UVOT_u

,time,flux,fluxerr,mag,magerr,mag_unc_u,mag_unc_l,lim,band,telescope,system,filter
2,59716.366188,183.5487,17.56941,18.240687,0.103927,0.099250,0.109244,0,uvot::u,Swift/UVOT,AB,$u$
8,59720.060356,342.6840,18.56192,17.562831,0.058810,0.057273,0.060463,0,uvot::u,Swift/UVOT,AB,$u$
14,59720.458080,290.8180,16.90595,17.741012,0.063116,0.061350,0.065025,0,uvot::u,Swift/UVOT,AB,$u$
20,59722.743700,334.1737,16.53143,17.590135,0.053711,0.052425,0.055085,0,uvot::u,Swift/UVOT,AB,$u$


## ATLAS

In [41]:
# joj_ATLAS = pd.read_csv("./atlas_forced_phot.txt", delimiter=" ", names=["MJD", "m", "dm", "uJy", "duJy", "F", "err", "chi/N", "RA", "Dec", "x", "y", "maj", "min", "phi", "apfit", "mag5sig", "Sky", "Obs"])
joj_ATLAS = np.loadtxt("./atlas_forced_phot.txt", dtype=object)
# joj_ATLAS = joj_ATLAS[:, [0, 2, 3, ]]
joj_ATLAS = joj_ATLAS[:, [0, 1, 2, 3, 4, 5]]
joj_ATLAS[:, :-1] = np.array(joj_ATLAS[:, :-1], dtype=float)
joj_ATLAS_o = joj_ATLAS[joj_ATLAS[:, -1] == "o"]
# arg = ((joj_ATLAS_o[:, 0] - t_max_mjd) / (1 + z) <= 30) & ((joj_ATLAS_o[:, 0] - t_max_mjd) / (1 + z) >= -8)
joj_ATLAS_o[:, -1] = "atlaso"
joj_ATLAS_o = joj_ATLAS_o#[arg]
joj_ATLAS_o = pd.DataFrame(joj_ATLAS_o, columns=["time", "mag", "magerr", "flux", "fluxerr", "band"])

joj_ATLAS_c = joj_ATLAS[joj_ATLAS[:, -1] == "c"]
# arg = ((joj_ATLAS_o[:, 0] - t_max_mjd) / (1 + z) <= 30) & ((joj_ATLAS_o[:, 0] - t_max_mjd) / (1 + z) >= -8)
joj_ATLAS_c[:, -1] = "atlasc"
joj_ATLAS_c = joj_ATLAS_c#[arg]
joj_ATLAS_c = pd.DataFrame(joj_ATLAS_c, columns=["time", "mag", "magerr", "flux", "fluxerr", "band"])
joj_ATLAS_c

joj_ATLAS_o["filter"] = "$o$"
joj_ATLAS_c["filter"] = "$c$"
joj_ATLAS_o["telescope"] = "ATLAS"
joj_ATLAS_c["telescope"] = "ATLAS"
joj_ATLAS_o["system"] = "AB"
joj_ATLAS_c["system"] = "AB"
joj_ATLAS_o

,time,mag,magerr,flux,fluxerr,band,filter,telescope,system
0,59650.590679,-23.539,9.167,-1.0,13.0,atlaso,$o$,ATLAS,AB
1,59650.594137,21.981,2.207,6.0,13.0,atlaso,$o$,ATLAS,AB
2,59650.624278,20.402,0.539,25.0,14.0,atlaso,$o$,ATLAS,AB
3,59650.640394,21.809,1.868,7.0,13.0,atlaso,$o$,ATLAS,AB
4,59654.532257,-19.952,0.625,-38.0,24.0,atlaso,$o$,ATLAS,AB
...,...,...,...,...,...,...,...,...,...
271,59796.79148,18.984,0.398,93.0,37.0,atlaso,$o$,ATLAS,AB
272,59796.801295,18.924,0.36,98.0,35.0,atlaso,$o$,ATLAS,AB
273,59799.012854,19.937,0.771,38.0,30.0,atlaso,$o$,ATLAS,AB
274,59799.016008,18.6,0.233,132.0,31.0,atlaso,$o$,ATLAS,AB


## KAIT \& Nickel BVRI

In [52]:
joj_UCB = pd.read_csv("./UCB_phot.dat", comment="#", sep="\t")
joj_UCB["telescope"] = np.append(["KAIT"] * 20, ["Nickel"] * 7)
joj_UCB["system"] = "Vega"
joj_UCB_flt = []
zp = [4063, 3636, 3064, 2416, 3000]
for k in range(5):
    flt = ["B", "V", "R", "I", "CLEAR"][k]
    fltname = ["b", "v", "r", "i", "clear"][k]
    joj_UCB_flt.append(
        pd.DataFrame(
            {
                "time": joj_UCB["MJD"],
                "band": ["bessell" + fltname] * len(joj_UCB),
                "mag": joj_UCB[flt],
                "magerr": joj_UCB["E" + flt],
                "flux": 10 ** (-0.4 * joj_UCB[flt]) * zp[k] * 1e6,
                # "fluxerr": (
                #     10 ** (-0.4 * (joj_UCB[flt] - joj_UCB["E" + flt]))
                #     - 10 ** (-0.4 * (joj_UCB[flt] + joj_UCB["E" + flt]))
                # )
                # * zp[k]
                # * 1e6
                # / 2,
                "fluxerr": 10 ** (-0.4 * joj_UCB[flt])
                * zp[k]
                * 1e6
                * (np.log(10) / 2.5)
                * joj_UCB["E" + flt],
                "telescope": joj_UCB["telescope"],
                "system": joj_UCB["system"],
            }
        )[~np.isnan(joj_UCB[flt])]
    )

joj_UCB_B, joj_UCB_V, joj_UCB_R, joj_UCB_I, joj_UCB_CLEAR = joj_UCB_flt
joj_UCB_B["filter"], joj_UCB_V["filter"], joj_UCB_R["filter"], joj_UCB_I["filter"], joj_UCB_CLEAR["filter"] = (
    "$B$",
    "$V$",
    "$R$",
    "$I$",
    "clear"
)
joj_UCB_CLEAR

,time,band,mag,magerr,flux,fluxerr,telescope,system,filter
0,59711.247,bessellclear,17.387,0.062,332.905718,19.010285,KAIT,Vega,clear
1,59712.422,bessellclear,17.082,0.072,440.880870,29.236773,KAIT,Vega,clear
2,59717.413,bessellclear,16.192,0.046,1000.739990,42.398917,KAIT,Vega,clear
3,59720.409,bessellclear,15.995,0.021,1199.834249,23.206852,KAIT,Vega,clear
4,59722.405,bessellclear,15.931,0.021,1272.685965,24.615929,KAIT,Vega,clear
5,59723.398,bessellclear,15.914,0.019,1292.769968,22.623018,KAIT,Vega,clear
6,59724.396,bessellclear,15.963,0.017,1235.723355,19.348436,KAIT,Vega,clear
7,59726.394,bessellclear,15.984,0.022,1212.052006,24.559505,KAIT,Vega,clear
9,59731.378,bessellclear,16.239,0.019,958.343617,16.770675,KAIT,Vega,clear
10,59732.375,bessellclear,16.282,0.020,921.130695,16.967854,KAIT,Vega,clear


## KAIT SDSS

In [72]:
joj_Fritz = pd.read_csv("./Fritz_phot.csv")
joj_KAIT = joj_Fritz[(joj_Fritz["instrument_name"] == "KAIT") & (~np.isnan(joj_Fritz["mag"]))]
joj_KAIT["time"] = joj_KAIT["mjd"]
joj_KAIT["system"] = "AB"
joj_KAIT["telescope"] = "KAIT"
KAIT_flt = ["SDSS "] * len(joj_KAIT)
for k in range(len(joj_KAIT)):
    KAIT_flt[k] += rf"${joj_KAIT['filter'].values[k].split('sdss')[-1]}$"
joj_KAIT["filter"] = KAIT_flt
joj_KAIT

/var/folders/_g/qsrvvtv17zn3wk8jrrrfhmx80000gn/T/ipykernel_45671/2645726550.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joj_KAIT["time"] = joj_KAIT["mjd"]
/var/folders/_g/qsrvvtv17zn3wk8jrrrfhmx80000gn/T/ipykernel_45671/2645726550.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joj_KAIT["system"] = "AB"
/var/folders/_g/qsrvvtv17zn3wk8jrrrfhmx80000gn/T/ipykernel_45671/2645726550.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

,obj_id,ra,dec,filter,mjd,snr,instrument_id,instrument_name,ra_unc,dec_unc,...,created_at,annotations,mag,magerr,magsys,limiting_mag,Delete,time,system,telescope
384,ZTF22aajijjf,NaN,NaN,SDSS $g$,59754.3175,15.879156,68,KAIT,NaN,NaN,...,2022-06-24T07:46:31.506599,NaN,18.206820,0.068375,ab,19.461464,NaN,59754.3175,AB,KAIT
385,ZTF22aajijjf,NaN,NaN,SDSS $r$,59754.3197,22.860334,68,KAIT,NaN,NaN,...,2022-06-24T07:47:58.310760,NaN,17.264332,0.047494,ab,18.914614,NaN,59754.3197,AB,KAIT
386,ZTF22aajijjf,NaN,NaN,SDSS $i$,59754.3206,21.656709,68,KAIT,NaN,NaN,...,2022-06-24T07:49:18.826196,NaN,17.616340,0.050134,ab,19.207897,NaN,59754.3206,AB,KAIT
393,ZTF22aajijjf,NaN,NaN,SDSS $g$,59756.3115,24.795233,68,KAIT,NaN,NaN,...,2022-06-26T07:37:34.148649,NaN,18.191386,0.043788,ab,19.929881,NaN,59756.3115,AB,KAIT
394,ZTF22aajijjf,NaN,NaN,SDSS $r$,59756.3138,22.703067,68,KAIT,NaN,NaN,...,2022-06-26T07:39:02.371552,NaN,17.360152,0.047823,ab,19.002938,NaN,59756.3138,AB,KAIT
395,ZTF22aajijjf,NaN,NaN,SDSS $i$,59756.3146,21.923743,68,KAIT,NaN,NaN,...,2022-06-26T07:40:23.419655,NaN,17.626543,0.049523,ab,19.231405,NaN,59756.3146,AB,KAIT
427,ZTF22aajijjf,NaN,NaN,SDSS $r$,59766.2868,14.652458,68,KAIT,NaN,NaN,...,2022-07-06T07:00:12.140915,NaN,17.900589,0.074099,ab,19.067940,NaN,59766.2868,AB,KAIT
450,ZTF22aajijjf,NaN,NaN,SDSS $r$,59777.2557,10.451560,68,KAIT,NaN,NaN,...,2022-07-17T06:17:48.896578,NaN,18.229432,0.103883,ab,19.029960,NaN,59777.2557,AB,KAIT


## LT

In [79]:
with open("./LT_phot.dat", "r") as f:
    lines = np.array(f.readlines())
flt = lines[1::6]

joj_LT = pd.DataFrame({"time": np.array(lines[2::6], dtype=float),
          "mag": np.array(lines[3::6], dtype=float),
          "magerr": np.array(lines[4::6], dtype=float),
          })
joj_LT["system"] = "AB"
joj_LT["telescope"] = "LT"

LT_flt = ["SDSS "] * len(joj_LT)
for k in range(len(joj_LT)):
    band = flt[k].split('sdss')[-1].split('\n')[0]
    LT_flt[k] += rf"${band}$"
joj_LT["filter"] = LT_flt

joj_LT

,time,mag,magerr,system,telescope,filter
0,59745.9109,17.498380,0.033776,AB,LT,SDSS $g$
1,59734.9548,16.562812,0.042253,AB,LT,SDSS $g$
2,59729.9460,16.154138,0.040814,AB,LT,SDSS $g$
3,59725.0830,15.918042,0.037055,AB,LT,SDSS $g$
4,59734.9559,16.546376,0.098408,AB,LT,SDSS $r$
5,59729.9471,16.177345,0.087092,AB,LT,SDSS $r$
6,59725.0841,16.072399,0.091686,AB,LT,SDSS $r$
7,59745.9119,16.909212,0.080901,AB,LT,SDSS $r$
8,59725.0851,16.767782,0.085107,AB,LT,SDSS $i$
9,59729.9481,17.027265,0.083928,AB,LT,SDSS $i$


## Tex table

In [81]:
joj_phot = pd.concat(
    [
        joj_ZTF_tex,
        joj_UVOT_u,
        joj_UVOT_b,
        joj_UVOT_v,
        joj_UVOT_uvw1,
        joj_UVOT_uvw2,
        joj_UVOT_uvm2,
        joj_ATLAS_o,
        joj_ATLAS_c,
        joj_UCB_B,
        joj_UCB_V,
        joj_UCB_R,
        joj_UCB_I,
        joj_UCB_CLEAR,
        joj_KAIT,
        joj_LT,
    ], join="inner"
)

joj_phot[(joj_phot["time"] >= 59707) & (joj_phot["mag"] > 0)]

,time,mag,magerr,filter,telescope,system
1006,59710.295278,18.2965,0.054465,ZTF $g$,P48/ZTF,AB
1007,59711.235474,17.62649,0.038705,ZTF $g$,P48/ZTF,AB
1008,59711.236412,17.672872,0.040479,ZTF $g$,P48/ZTF,AB
1011,59712.287639,17.123556,0.037656,ZTF $g$,P48/ZTF,AB
1015,59717.332593,16.093082,0.030337,ZTF $g$,P48/ZTF,AB
...,...,...,...,...,...,...
11,59745.913,17.421906,0.087728,SDSS $i$,LT,AB
12,59729.9492,16.979238,0.110688,SDSS $z$,LT,AB
13,59745.914,17.147068,0.097712,SDSS $z$,LT,AB
14,59725.0862,16.734513,0.106353,SDSS $z$,LT,AB


In [82]:
arg = ((joj_phot["time"] - t_max_mjd) / (1 + z) <= 80) & (
    (joj_phot["time"] - t_max_mjd) / (1 + z) >= -19
)
joj_phot = Table(dict(joj_phot[arg]))

joj_phot.sort("time")

joj_phot = joj_phot[(joj_phot["time"] >= 59707) & (joj_phot["mag"] > 0)]

joj_phot["time"] = np.round(np.array(joj_phot["time"], dtype=float), decimals=3)
joj_phot["mag"] = np.round(np.array(joj_phot["mag"], dtype=float), decimals=3)
joj_phot["magerr"] = np.round(np.array(joj_phot["magerr"], dtype=float), decimals=3)
# joj_phot["flux"] = np.array(
#     np.round(np.array(joj_phot["flux"], dtype=float), decimals=1), dtype=object
# )
# joj_phot["flux"][joj_phot["telescope"] == "ATLAS"] = np.array(
#     joj_phot["flux"][joj_phot["telescope"] == "ATLAS"], dtype=int
# )
# joj_phot["fluxerr"] = np.array(
#     np.round(np.array(joj_phot["fluxerr"], dtype=float), decimals=1), dtype=object
# )
# joj_phot["fluxerr"][joj_phot["telescope"] == "ATLAS"] = np.array(
#     joj_phot["fluxerr"][joj_phot["telescope"] == "ATLAS"], dtype=int
# )

from astropy.io import ascii

ascii.write(
    joj_phot["time", "filter", "mag", "magerr", "system", "telescope"],
    "./phot_table.dat",
    Writer=ascii.Latex,
    latexdict={"tabletype": "table*"},
    overwrite=True,
)

## SALT3 fit

In [ ]:
joj_SALT = {}
lc = joj_ZTF.copy()
arg = (joj_ZTF['phase'] <= 150) & (joj_ZTF['phase'] >= -25)
joj_SALT["time"] = lc["MJD"][arg]
joj_SALT["band"] = lc["passband"][arg]
joj_SALT["flux"] = lc["fnu_microJy"][arg]
joj_SALT["fluxerr"] = lc["fnu_microJy_unc"][arg]
joj_SALT = pd.concat(
    [
        pd.DataFrame(joj_SALT),
        joj_UVOT_b,
        joj_UVOT_v,
        joj_ATLAS_o,
        joj_ATLAS_c,
        joj_UCB_B,
        joj_UCB_V,
        joj_UCB_R,
        joj_UCB_I,
    ]
)

joj_SALT["zp"] = [2.5 * np.log10(3631 * 1e6)] * len(joj_SALT["time"])
joj_SALT["zpsys"] = ["ab"] * len(joj_SALT["time"])
arg = ((joj_SALT["time"] - t_max_mjd) / (1 + z) <= 20) & (
    (joj_SALT["time"] - t_max_mjd) / (1 + z) >= -10
)

joj_SALT = Table(dict(joj_SALT[arg]))
# joj_SALT

In [ ]:
import sncosmo

model = sncosmo.Model(
    source="salt3",
    effects=[sncosmo.F99Dust()],
    effect_names=["mw"],
    effect_frames=["obs"],
)
model.set(z=z, mwebv=ebv)

np.random.seed(321)
result, fitted_model = sncosmo.mcmc_lc(
    joj_SALT, model, ["t0", "x0", "x1", "c"], nburn=1000, nsamples=500, thin=5
)

_ = sncosmo.plot_lc(joj_SALT, model=fitted_model, errors=result.errors, xfigsize=12)

In [ ]:
leff_B = 4354.45  # AA
leff_V = 5336.14  # AA
leff_g = 4722.74  # AA
leff_r = 6339.61  # AA
leff_i = 7886.13  # AA
leff_uvot_u = 3523.78
leff_uvot_b = 4345.96
leff_uvot_v = 5412.38
leff_atlas_o = 6629.82
rv = 3.1
ebv = 0.032

In [ ]:
from copy import deepcopy

samples = result.samples
sample_model = deepcopy(model)
t_max_s = np.empty((len(samples), 7))
m_max_s = np.empty((len(samples), 7))
for k, sample in enumerate(samples):
    sample_model.parameters[:] = np.append(np.append(z, sample), ebv)
    for j, (flt, leff) in enumerate(
        zip(
            ["bessellb", "ztfg", "ztfr", "ztfi", "atlaso", "uvot::b", "uvot::u"],
            [leff_B, leff_g, leff_r, leff_i, leff_atlas_o, leff_uvot_b, leff_uvot_u],
        )
    ):
        peakphase = sample_model.source.peakphase(flt)
        t_max_s[k, j] = peakphase * (1 + z) + sample[0]
        m_max_s[k, j] = (
            sample_model.source.bandmag(flt, "ab", peakphase)
            - mu
            - calALambda(wv=leff, RV=3.1, EBV=ebv)
        )

In [ ]:
(
    joj_ZTF_g[(joj_ZTF_g["MJD"] > 59720) & (joj_ZTF_g["MJD"] < 59725)]["mag"]
    - mu
    - calALambda(wv=leff_g, RV=3.1, EBV=ebv)
), (joj_ZTF_g[(joj_ZTF_g["MJD"] > 59720) & (joj_ZTF_g["MJD"] < 59725)]["mag_err_u"]), (
    joj_ZTF_r[(joj_ZTF_r["MJD"] > 59720) & (joj_ZTF_r["MJD"] < 59725)]["mag"]
    - mu
    - calALambda(wv=leff_r, RV=3.1, EBV=ebv)
), (
    joj_ZTF_r[(joj_ZTF_r["MJD"] > 59720) & (joj_ZTF_r["MJD"] < 59725)]["mag_err_u"]
)

In [ ]:
joj_UVOT_b.loc[19]["mag"] - mu - calALambda(wv=leff_uvot_b, RV=3.1, EBV=ebv)

In [ ]:
for j, (flt, leff) in enumerate(
    zip(
        ["bessellb", "ztfg", "ztfr", "ztfi", "atlaso", "uvot::b", "uvot::u"],
        [leff_B, leff_g, leff_r, leff_i, leff_atlas_o, leff_uvot_b, leff_uvot_u],
    )
):
    print(flt)
    t_max, t_max_lo, t_max_up = (
        np.percentile(t_max_s[:, j], 50),
        np.percentile(t_max_s[:, j], 16),
        np.percentile(t_max_s[:, j], 84),
    )
    m_max, m_max_lo, m_max_up = (
        np.percentile(m_max_s[:, j], 50),
        np.percentile(m_max_s[:, j], 16),
        np.percentile(m_max_s[:, j], 84),
    )
    print(
        "t_max (MJD) = {:.2f} + {:.2f} - {:.2f}".format(
            t_max, t_max_up - t_max, t_max - t_max_lo
        )
    )
    print(
        "M_max (mag) = {:.4f} + {:.4f} - {:.4f}".format(
            m_max, m_max_up - m_max, m_max - m_max_lo
        )
    )

t_max_mjd_B = 59723.1